In [8]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import undetected_chromedriver as uc    
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [ ]:
browser = uc.Chrome(options=options)


In [6]:
url='https://servicios.pami.org.ar/vademecum/views/consultaPublica/listado.zul'


In [ ]:
browser.get(url)

In [47]:
time.sleep(3)
input_element = browser.find_element(By.ID, 'zk_comp_40-real').click()
time.sleep(3)
input_element = browser.find_element(By.ID, 'zk_comp_133-cave').click()
time.sleep(3)
input_element = browser.find_element(By.ID, 'zk_comp_80').click()


In [48]:
all_data = []

In [ ]:
try:
    # Espera hasta que el div específico esté presente
    div_element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.ID, 'zk_comp_98'))
    )

    # Dentro de este div, encuentra el span con la clase 'z-paging-text'
    span_element = div_element.find_element(By.CLASS_NAME, 'z-paging-text')

    # Extrae el texto del span
    span_text = span_element.text

    # Usa una expresión regular para obtener solo el número
    match = re.search(r'\d+', span_text)
    if match:
        number = match.group()
        print(f'El número es: {number}')
    

except Exception as e:
    print(f'Error: {e}')

# Especifica el número total de páginas a iterar
num_pages =  int(number) # Reemplaza con el número real de páginas que necesitas iterar

for _ in range(num_pages):
    # Esperar hasta que la tabla esté presente
    wait = WebDriverWait(browser, 10)
    wait.until(EC.presence_of_element_located((By.ID, 'zk_comp_86-cave')))

    # Obtener el contenido de la página actual
    html = browser.page_source
    soup = bs(html, 'lxml')
    table = soup.find('table', {'id': 'zk_comp_86-cave'})
    rows = table.find_all('tr')

    # Extraer los datos de la tabla
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append(cols)

    # Agregar los datos extraídos a la lista principal
    all_data.extend(data)

    # Intentar encontrar el botón para pasar a la siguiente página
    try:
        next_button = browser.find_element(By.NAME, 'zk_comp_98-next')
        next_button.click()
        time.sleep(3)
    except Exception as e:
        print(e)
        break  # Salir del bucle si no se puede hacer clic en el botón de siguiente página

# Mostrar todos los datos extraídos
for item in all_data:
    print(item)

In [ ]:
filtered_data = [row for row in all_data if 'No se han encontrado resultados' not in row]

columns = ['Indice', 'N° Certificado', 'Laboratorio','Nombre Comercial','Forma Farmacéutica','Presentación','Detalle','Genérico','Disponibilidad','Envase Secundario']
df = pd.DataFrame(filtered_data, columns=columns)
df2 = df.drop(columns=['Indice','Disponibilidad','Envase Secundario','Detalle'])
df2

In [54]:
df2.to_csv('papa.csv', index=False, encoding='utf-8-sig')

In [ ]:
import csv
# Nombre del archivo CSV que deseas crear
csv_file = 'test.csv'

# Escribir los datos en el archivo CSV
with open(csv_file, 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['Envase Secundario', 'N° Certificado', 'Laboratorio','Nombre Comercial','Forma Farmacéutica','Presentación','Detalle','Genérico','Disponibilidad'])

print(f'Se han guardado los datos en "{csv_file}"')